# Pipeline the creación the archivos para ingesta de datos en Dataflow "metadata" set

## Pasos

0. Importar librerias
1. paso 1: Importar archivo "category_final"
2. Paso 2: Concatenación de todos los archivos Json del set "metadata"
3. Paso 3: Creacion de un dataframe copia del original
4. Paso 4: Eliminar caracteres
5. Paso 5: Filtrado del Set "metadata"
6. Paso 6: Eliminación de duplicado por "gmap_id"
7. Paso 7: Creacion de un .csv para filtrar el Set "reviews"
8. Exportar datos en Json


### paso 0: Importar librerias

In [2]:
import pandas as pd

### paso 1: Importar archivo "category_final"
Este archivo lo utilizaremos en el datafrema "categorias_con_filtro" para filtrar toda la data que no corresponde a las mismas

In [3]:
categorias_con_filtro = pd.read_csv("./Dataset/auxi/category_final.csv")

In [4]:
categorias_con_filtro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  534 non-null    object
dtypes: object(1)
memory usage: 4.3+ KB


### Paso 2: Concatenación de todos los archivos Json del set "metadata" en un unico dataframe llamado "df_metadata_all"

In [5]:
df_metadata_all = pd.DataFrame()

for i in range(1, 12):
    filename = f'./Dataset/Google_Maps/metadata-sitios/metadata-sitios/{i}.json'
    df = pd.read_json(filename, lines=True)
    df_metadata_all = pd.concat([df_metadata_all, df], ignore_index=True)

df_metadata_all

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,[Fabric store],3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Service options': ['In-store shopping'], 'Pa...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,[Fabric store],4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",{'Service options': ['In-store pickup']},Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025006,Steak 48,"Steak 48, 260 S Broad St, Philadelphia, PA 19102",0x89c6c74f43a49b55:0x6be6995921c58b12,None,39.947254,-75.164953,"[Steak house, Fine dining restaurant, Seafood ...",4.6,308,None,"[[Monday, 4–10PM], [Tuesday, 4–10PM], [Wednesd...","{'Service options': ['Takeout', 'Dine-in', 'De...",Closed ⋅ Opens 4PM,"[0x89c6c7af1a55f4f9:0x589bb93addf7fd95, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...
3025007,"Jack Mcnerney Chevrolet, Inc.","Jack Mcnerney Chevrolet, Inc., 363 NY-281, Tul...",0x89da1787d3fdc4a5:0xce92ea6e3cd4d1cc,None,42.788636,-76.122120,[Chevrolet dealer],4.4,143,None,"[[Monday, 8:30AM–7PM], [Tuesday, 8:30AM–7PM], ...","{'Service options': ['In-store shopping'], 'Of...",Open ⋅ Closes 7PM,"[0x89d98d228469e435:0x3fdc4fa72594151, 0x89d9e...",https://www.google.com/maps/place//data=!4m2!3...
3025008,Central Ny Spay Neuter Assista,"Central Ny Spay Neuter Assista, 17 Salisbury S...",0x89da6b9ce832726f:0xbd0d3630f1e3b42e,None,42.603831,-76.165286,[Veterinarian],4.3,88,None,None,{'Planning': ['Appointments recommended']},Open now,"[0x89da6c67512700cb:0x69892b3a9a648efe, 0x89d9...",https://www.google.com/maps/place//data=!4m2!3...
3025009,Ok Feed Store,"Ok Feed Store, 16300 SW 296th St, Homestead, F...",0x88d9e872a9e39f3d:0x5de29f50507e5d73,None,25.491641,-80.456076,[Animal feed store],4.9,13,None,"[[Monday, 9AM–6PM], [Tuesday, 9AM–6PM], [Wedne...","{'Service options': ['Curbside pickup', 'In-st...",Open ⋅ Closes 6PM,"[0x88d9e86cb955da7b:0xeda838b28926c524, 0x88d9...",https://www.google.com/maps/place//data=!4m2!3...


### Paso 3: Creacion de un dataframe copia del original para no contaminar nuestro set RAIZ

In [6]:
df_metadata_tras=df_metadata_all

### Paso 4: Eliminar caracteres
En este paso eliminamos corchetes en nuestra columna "category":
- Transformamos el DataType de Object a String
- Sacamos el primer parentecis con el metodo .str.replace
- Sacamos el segundo parentecis con el metodo .str.replace

nota: Este paso es necesario para poder utilizar nuestro filtro de categorias de manera correcta

In [7]:

df_metadata_tras['category'] = df_metadata_tras['category'].astype(str)
df_metadata_tras['category'] = df_metadata_tras['category'].str.replace("[", "").str.replace("]", "")
df_metadata_tras['category'] = df_metadata_tras['category'].str.replace("'", "").str.replace("'", "")
df_metadata_tras

C:\Users\miauchis\AppData\Local\Temp\ipykernel_17956\708717600.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_metadata_tras['category'] = df_metadata_tras['category'].str.replace("[", "").str.replace("]", "")


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,Pharmacy,4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,Textile exporter,4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,Korean restaurant,4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,Fabric store,3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Service options': ['In-store shopping'], 'Pa...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,Fabric store,4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",{'Service options': ['In-store pickup']},Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025006,Steak 48,"Steak 48, 260 S Broad St, Philadelphia, PA 19102",0x89c6c74f43a49b55:0x6be6995921c58b12,None,39.947254,-75.164953,"Steak house, Fine dining restaurant, Seafood r...",4.6,308,None,"[[Monday, 4–10PM], [Tuesday, 4–10PM], [Wednesd...","{'Service options': ['Takeout', 'Dine-in', 'De...",Closed ⋅ Opens 4PM,"[0x89c6c7af1a55f4f9:0x589bb93addf7fd95, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...
3025007,"Jack Mcnerney Chevrolet, Inc.","Jack Mcnerney Chevrolet, Inc., 363 NY-281, Tul...",0x89da1787d3fdc4a5:0xce92ea6e3cd4d1cc,None,42.788636,-76.122120,Chevrolet dealer,4.4,143,None,"[[Monday, 8:30AM–7PM], [Tuesday, 8:30AM–7PM], ...","{'Service options': ['In-store shopping'], 'Of...",Open ⋅ Closes 7PM,"[0x89d98d228469e435:0x3fdc4fa72594151, 0x89d9e...",https://www.google.com/maps/place//data=!4m2!3...
3025008,Central Ny Spay Neuter Assista,"Central Ny Spay Neuter Assista, 17 Salisbury S...",0x89da6b9ce832726f:0xbd0d3630f1e3b42e,None,42.603831,-76.165286,Veterinarian,4.3,88,None,None,{'Planning': ['Appointments recommended']},Open now,"[0x89da6c67512700cb:0x69892b3a9a648efe, 0x89d9...",https://www.google.com/maps/place//data=!4m2!3...
3025009,Ok Feed Store,"Ok Feed Store, 16300 SW 296th St, Homestead, F...",0x88d9e872a9e39f3d:0x5de29f50507e5d73,None,25.491641,-80.456076,Animal feed store,4.9,13,None,"[[Monday, 9AM–6PM], [Tuesday, 9AM–6PM], [Wedne...","{'Service options': ['Curbside pickup', 'In-st...",Open ⋅ Closes 6PM,"[0x88d9e86cb955da7b:0xeda838b28926c524, 0x88d9...",https://www.google.com/maps/place//data=!4m2!3...


### Paso 5: Filtrado del Set "metadata" con nuestras categorias
Este paso utilisa una mascara sobre el dataframe "df_metadata_tras" filtrando todas las categorias ausentes en el dataframe "categorias_con_filtro" utilizando la columna "category"

nota: En este paso tambien comprobamos como la data decrece aproximadamente a un 10% de la data original.
(predicción hecha por nuestro departamento de DS)

In [8]:
#filtro de df_metadata por categorias
mask = df_metadata_tras['category'].isin(categorias_con_filtro['category'])


df_metadata_tras = df_metadata_tras[mask]
df_metadata_tras.shape

(331966, 15)

### Paso 6: Eliminación de duplicado por "gmap_id"

In [9]:
#drop duplicated
df_metadata_tras.drop_duplicates(subset='gmap_id', inplace=True)
df_metadata_tras.shape

C:\Users\miauchis\AppData\Local\Temp\ipykernel_17956\741094409.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metadata_tras.drop_duplicates(subset='gmap_id', inplace=True)


(329537, 15)

In [10]:
df_metadata_tras

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,Korean restaurant,4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
6,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,None,33.916402,-118.010855,Restaurant,4.5,18,None,"[[Thursday, 11AM–9:30PM], [Friday, 11AM–9:30PM...","{'Service options': ['Outdoor seating', 'Curbs...",Open ⋅ Closes 9:30PM,None,https://www.google.com/maps/place//data=!4m2!3...
12,Lena’s Beauty Nails Lashes Brows,"Lena’s Beauty Nails Lashes Brows, 657 NE Ventu...",0x87ec2481df6626b5:0xff5627745308ec2b,None,41.616310,-93.858345,Beauty salon,4.2,128,None,"[[Thursday, 9AM–7PM], [Friday, 9AM–7PM], [Satu...","{'Health & safety': ['Appointment required'], ...",Open ⋅ Closes 7PM,"[0x87ec20ce836cac57:0x83bd66577e9a7a32, 0x87ee...",https://www.google.com/maps/place//data=!4m2!3...
43,Sigman’s Diner,"Sigman’s Diner, 517 Elk St, Gassaway, WV 26624",0x884bdf439ee0d6e3:0x16676d0a5982954d,None,38.672489,-80.774893,Diner,4.6,28,None,"[[Thursday, 7AM–2PM], [Friday, 7AM–2PM], [Satu...","{'Service options': ['Takeout', 'Dine-in', 'De...",Permanently closed,[0x884bcf5834725e7d:0x537d8a1a05e15936],https://www.google.com/maps/place//data=!4m2!3...
66,The Beauty Bar,"The Beauty Bar, 413 Lincoln St, Roseville, CA ...",0x809b2037e088215d:0xbd3cebbdb6533638,None,38.752340,-121.285424,Beauty salon,4.9,21,None,"[[Thursday, 9AM–8PM], [Friday, 9AM–8PM], [Satu...","{'Health & safety': ['Mask required', 'Staff w...",Open ⋅ Closes 8PM,"[0x809b2037fa451d37:0x15f7ae418a9c671f, 0x809b...",https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3024964,Market Cafe,"Market Cafe, 2312 AR-367, Bald Knob, AR 72010",0x87d3a24c6b9f6b1b:0x9ea26cbe6dc7aad9,None,35.306025,-91.574052,American restaurant,4.4,178,$,"[[Monday, 6AM–8PM], [Tuesday, 6AM–8PM], [Wedne...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 8PM,"[0x87d3a3b2fecb8a8b:0x6198f40100abdb31, 0x87d3...",https://www.google.com/maps/place//data=!4m2!3...
3024970,Las Playitas,"Las Playitas, 3001 AR-25B, Heber Springs, AR 7...",0x87d247cfd66dd153:0x915c7425736e865a,None,35.515762,-92.019176,Mexican restaurant,4.3,154,$,"[[Monday, 11AM–9:30PM], [Tuesday, 11AM–9:30PM]...","{'Service options': ['Curbside pickup', 'No-co...",Closed ⋅ Opens 11AM,None,https://www.google.com/maps/place//data=!4m2!3...
3024974,La Casa Mexican Restaurant,"La Casa Mexican Restaurant, 14 Bee Branch Rd, ...",0x87d268b0fbbfd4b9:0xeec251970fae404c,None,35.384414,-92.215903,Mexican restaurant,4.5,76,$$,"[[Monday, 11AM–7PM], [Tuesday, 11AM–7PM], [Wed...","{'Service options': ['Takeout', 'Dine-in', 'De...",Permanently closed,"[0x87d279d0d5c14cc3:0xf0f3164d7467bd5, 0x87d24...",https://www.google.com/maps/place//data=!4m2!3...
3024976,Country Kitchen,"Country Kitchen, 701 Main St, Pangburn, AR 72121",0x87d24cd30b7695d1:0xdc25418ff311a6aa,Homey chain eatery serving breakfast all day &...,35.425678,-91.839200,American restaurant,4.5,78,$,"[[Monday, 6AM–2PM], [Tuesday, 6AM–2PM], [Wedne...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 2PM,"[0x87d24cd14c2328ff:0x42797c7e42cd07ba, 0x87d2...",https://www.google.com/maps/place//data=!4m2!3...


In [11]:
duplicates = df_metadata_tras.duplicated(subset='gmap_id').sum()
print("Number of duplicate gmap_id: ", duplicates)


Number of duplicate gmap_id:  0


### Paso 7: Creacion de un .csv que contiene los datos de nuestros negocios target para realizar el filtro de categorias de nuestro set "reviews"
Este paso es el que nos propiorciona la mascara para el filtro de las reviews de nuestros locales target

In [12]:
# export gmap_id in csv
df_metadata_tras[['gmap_id']].to_csv("./Dataset/auxi/gmap_id_metadata_filtrado_category_final.csv", index=False)


### Paso 8: Exportar datos en Json para ingestion en la infraestructura de GCP

In [13]:
# Step 5: export data into state.json
df_metadata_tras.to_json("./Dataset/metadata_filtered/metadata_final", orient="records")